# Creating a simulated starfield image and performing photometry using `webbpsf` and `photutils.GriddedPSFModel`

The simulated image will be made using a spatially-dependent JWST point-spread function (PSF) generated by `webbpsf`.  `photutils` will be then used to perform PSF-fitting photometry on the simulated JWST NIRCam image.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm
from astropy.table import Table
from astropy.visualization import simple_norm

from webbpsf.gridded_library import display_psf_grid
from webbpsf.utils import to_griddedpsfmodel

from photutils.datasets import make_noise_image
from photutils import BasicPSFPhotometry
from photutils import DBSCANGroup, MMMBackground

%matplotlib inline

We start by loading into FITS HDU a pre-generated grid of JWST PSFs for NIRCam detector A1 in F090W.

In [ ]:
url = 'https://stsci.box.com/shared/static/6h8wsr2ts0t24s79cxmnyk8bldt0vv3i.fits'
hdu = fits.open(url)

We now create a `photutils.GriddedPSFModel` from the FITS HDU.

In [ ]:
psfmodel = to_griddedpsfmodel(hdu, ext=0)

In [ ]:
psfmodel

`psfmodel` is an instance of a `photutils.GriddedPSFModel`.  Let's view some of it's attributes.

The oversampled PSF grid is stored internally as a 3D `numpy.ndarray`.  The first axis represents the number of PSFs, while the second and third axes represent the (ny, nx) shape of the 2D PSFs.

Here there are 16 PSFs (from a 4x4 reference grid) and the shape of each is 404x404 pixels.

In [ ]:
psfmodel.data.shape

The `oversampling` attribute is the PSF oversampling factor.

In [ ]:
psfmodel.oversampling

The `grid_xypos` attribute contains a list of the `(x, y)` positions of reference PSFs.  The PSF at an arbitrary `(x, y)` position is interpolated from the grid of reference PSFs.

In [ ]:
psfmodel.grid_xypos

This model contains a 4x4 grid of reference PSFs.

In [ ]:
len(psfmodel.grid_xypos)

The `meta` attribute is dictionary holding detailed information how the PSF model grid was generated by `webbpsf`

In [ ]:
psfmodel.meta

We can use the `webbpsf.gridded_library.display_psf_grid` function to visualize the PSF grid.

In [ ]:
display_psf_grid(psfmodel)

Now let's use the `psfmodel` grid to create a simulated starfield image.
First, we'll define 1000 stars with random positions and fluxes.

In [ ]:
shape = (2047, 2047)
data = np.zeros(shape)
nstars = 1000
from astropy.utils.misc import NumpyRNGContext
with NumpyRNGContext(12345):    # seed for repeatability
    xx = np.random.uniform(low=0, high=shape[1], size=nstars)
    yy = np.random.uniform(low=0, high=shape[0], size=nstars)
    zz = np.random.uniform(low=0, high=200000., size=nstars)

Now we'll evaluate the model at these positions and fluxes.

In [ ]:
eval_xshape = np.int(np.ceil(psfmodel.data.shape[2] / psfmodel.oversampling))
eval_yshape = np.int(np.ceil(psfmodel.data.shape[1] / psfmodel.oversampling))

for xxi, yyi, zzi in zip(xx, yy, zz):
    x0 = np.int(np.floor(xxi - (eval_xshape - 1) / 2.))
    y0 = np.int(np.floor(yyi - (eval_yshape - 1) / 2.))
    x1 = x0 + eval_xshape
    y1 = y0 + eval_yshape
    
    if x0 < 0:
        x0 = 0
    if y0 < 0:
        y0 = 0
    if x1 > shape[1]:
        x1 = shape[1]
    if y1 > shape[0]:
        y1 = shape[0]
        
    y, x = np.mgrid[y0:y1, x0:x1]
    data[y, x] += psfmodel.evaluate(x=x, y=y, flux=zzi, x_0=xxi, y_0=yyi)

Let's add some noise to the image.

In [ ]:
noise = make_noise_image(data.shape, 'gaussian', mean=0, stddev=2, random_state=123)
data += noise

Let's display the simulated JWST NIRCam det A1 F090W image.

In [ ]:
norm = simple_norm(data, 'sqrt', percent=99.)
plt.figure(figsize=(15, 15))
plt.imshow(data, norm=norm, origin='lower')

### PSF-fitting photometry with `GriddPSFModel`

We can also use `GriddedPSFModel` to perform PSF-fitting photometry.
Let's start by creating another simulated image with only a few sources, which 
will enable the fitting to run much faster.

In [ ]:
shape = (2047, 2047)
data = np.zeros(shape)
nstars = 25
from astropy.utils.misc import NumpyRNGContext
with NumpyRNGContext(12345):    # seed for repeatability
    xx = np.random.uniform(low=0, high=shape[1], size=nstars)
    yy = np.random.uniform(low=0, high=shape[0], size=nstars)
    zz = np.random.uniform(low=0, high=200000., size=nstars)
    
eval_xshape = np.int(np.ceil(psfmodel.data.shape[2] / psfmodel.oversampling))
eval_yshape = np.int(np.ceil(psfmodel.data.shape[1] / psfmodel.oversampling))

for xxi, yyi, zzi in zip(xx, yy, zz):
    x0 = np.int(np.floor(xxi - (eval_xshape - 1) / 2.))
    y0 = np.int(np.floor(yyi - (eval_yshape - 1) / 2.))
    x1 = x0 + eval_xshape
    y1 = y0 + eval_yshape
    
    if x0 < 0:
        x0 = 0
    if y0 < 0:
        y0 = 0
    if x1 > shape[1]:
        x1 = shape[1]
    if y1 > shape[0]:
        y1 = shape[0]
        
    y, x = np.mgrid[y0:y1, x0:x1]
    data[y, x] += psfmodel.evaluate(x=x, y=y, flux=zzi, x_0=xxi, y_0=yyi)
    
noise = make_noise_image(data.shape, 'gaussian', mean=0, stddev=2, random_state=123)
data += noise

Let's view this new simulated image.

In [ ]:
norm = simple_norm(data, 'sqrt', percent=99.)
plt.figure(figsize=(15, 15))
plt.imshow(data, norm=norm, origin='lower')

For this simple example, we'll use `photutils.BasicPSFPhotometry` to perform the photometry.  Here we're inputting a table of the initial positions (instead of inputting a star finder object).

First, we'll create a table defining the initial guesses of star positions.

In [ ]:
init_tbl = Table()
init_tbl['x_0'] = xx.astype(int)
init_tbl['y_0'] = yy.astype(int)
init_tbl['flux_0'] = zz.astype(int)

We then define the parameters to initialize a `BasicPSFPhotometry` instance and then call the instance on the data.

In [ ]:
sigma_psf = 3.
daogroup = DBSCANGroup(2.0*sigma_psf*gaussian_sigma_to_fwhm)
mmm_bkg = MMMBackground()
phot = BasicPSFPhotometry(daogroup, mmm_bkg, psfmodel, (101, 101), finder=None, 
                          aperture_radius=3.)
tbl = phot(data, init_guesses=init_tbl)

The result is an astropy Table containing the initial and fitted values of the star
positions and fluxes 

In [ ]:
tbl

We can also view the residual image of the best-fit model PSF image subtracted from the data.

In [ ]:
plt.figure(figsize=(15, 15))
diff = phot.get_residual_image()
plt.imshow(diff, norm=norm, origin='lower')
plt.colorbar()

The residual image is essentially noise, indicating good PSF model fits to the data.